In [2]:
import pandas as pd
import numpy as np
import requests
import random

In [3]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

Solving environment: ...working... done

# All requested packages already installed.



In [4]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('done')

Solving environment: ...working... done

# All requested packages already installed.

done


In [5]:
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

In [6]:
CLIENT_ID = 'K14R3V3X3AVZFWCTVGBRX5L1EWGP3HXF4JUIYQ2THCKGG05V'
CLIENT_SECRET = 'MNKYRAOGIJZVYGWJYD44N0NQLYOLP0BO0NHZIGNDOXHHUU3L'
VERSION = '20200822'
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K14R3V3X3AVZFWCTVGBRX5L1EWGP3HXF4JUIYQ2THCKGG05V
CLIENT_SECRET:MNKYRAOGIJZVYGWJYD44N0NQLYOLP0BO0NHZIGNDOXHHUU3L


In [7]:
cities = ["New York, NY", 'Chicago, IL', 'Los Angeles, LA'  ]
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d10f941735") # Indian Restaurant category in Foursquare -- reference = https://developer.foursquare.com/docs/build-with-foursquare/categories/
    results[city] = requests.get(url).json()

In [8]:
df_venues={}
for city in cities:
    venues = pd.json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [9]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):  #adding markers here
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Indian restaurants in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of Indian restaurants in New York, NY =  246
Showing Top 100
Total number of Indian restaurants in Chicago, IL =  169
Showing Top 100
Total number of Indian restaurants in Los Angeles, LA =  233
Showing Top 100


Now, I will plot the maps of our three major cities which I have taken.

In [10]:
maps[cities[0]]

In [11]:
maps[cities[1]]

In [12]:
maps[cities[2]]

Hence, we can find the best Indian restaurants to hangout with our family.